<a href="https://colab.research.google.com/github/TheJv21/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

In [28]:
!pip install nltk
!pip install numpy
!pip install keras

In [29]:
import json
import pickle
import random

import nltk
import numpy as np
from keras.layers import Activation, Dense, Dropout
import tensorflow as tf


#Para crear la red neuronal
from keras.models import Sequential
#from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer  #Para pasar las palabras a su forma raíz

In [30]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/intent.json').read())

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '¿', '.', ',']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [31]:
#Clasifica los patrones y las categorías
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [32]:
#Pasa la información a unos y ceros según las palabras presentes en cada categoría para hacer el entrenamiento
training = []
output_empty = [0]*len(classes)
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
print(training)

[[list([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
  list([0, 0, 0, 0, 1, 0])]
 [list([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 1, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 0, 1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
  list([0, 0, 0, 1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 0, 1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0

<ipython-input-32-9517ee53e56a>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [33]:
#Reparte los datos para pasarlos a la red
train_x = list(training[:,0])
train_y = list(training[:,1])

In [34]:
#Creamos la red neuronal
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [35]:
#Creamos el optimizador y lo compilamos
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])


In [36]:
#Entrenamos el modelo y lo guardamos
train_process = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save("chatbot_model.h5", train_process)

Epoch 1/100
4/4 [==============================] - 0s 3ms/step - loss: 1.7348 - accuracy: 0.1667
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 1.8605 - accuracy: 0.1111
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 1.8496 - accuracy: 0.2778
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 1.7925 - accuracy: 0.2778
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 1.7580 - accuracy: 0.2222
Epoch 6/100
4/4 [==============================] - 0s 3ms/step - loss: 1.7575 - accuracy: 0.3333
Epoch 7/100
4/4 [==============================] - 0s 2ms/step - loss: 1.8621 - accuracy: 0.1111
Epoch 8/100
4/4 [==============================] - 0s 2ms/step - loss: 1.7564 - accuracy: 0.2222
Epoch 9/100
4/4 [==============================] - 0s 2ms/step - loss: 1.8080 - accuracy: 0.1111
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 1.7376 - accuracy: 0.1111
Epoch 11/100
4/4 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Chatbot

In [37]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from keras.models import load_model

lemmatizer = WordNetLemmatizer()

In [38]:
#Importamos los archivos generados en el código anterior
intents = json.loads(open('/content/intent.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [39]:
#Pasamos las palabras de oración a su forma raíz
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [40]:
#Convertimos la información a unos y ceros según si están presentes en los patrones
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i]=1
    print(bag)
    return np.array(bag)

In [41]:
#Predecimos la categoría a la que pertenece la oración
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    max_index = np.where(res ==np.max(res))[0][0]
    category = classes[max_index]
    return category

In [42]:
#Obtenemos una respuesta aleatoria
def get_response(tag, intents_json):
    list_of_intents = intents_json['intents']
    result = ""
    for i in list_of_intents:
        if i["tag"]==tag:
            result = random.choice(i['responses'])
            break
    return result

In [45]:
#Ejecutamos el chat en bucle
contador = 0
while contador==0:
    contador+=1
    message=input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

algo gracioso
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1/1 [==============================] - 0s 19ms/step
¿Por qué el camarero fue despedido? Porque siempre se equivocaba con los pedidos. Una vez, le preguntó a un cliente si quería un café y el cliente le dijo que sí. El camarero le preguntó qué tamaño quería y el cliente le dijo que grande. El camarero le preguntó si quería azúcar y el cliente le dijo que sí. El camarero le preguntó si quería leche y el cliente le dijo que sí. El camarero le preguntó si quería nata y el cliente le dijo que sí. El camarero le preguntó si quería una cuchara y el cliente le dijo que sí. El camarero le preguntó si quería un vaso y el cliente le dijo que sí. El camarero le preguntó si quería una servilleta y el cliente le dijo que sí. El camarero le preguntó si quería un plato y el cliente le dijo que sí. El camarero le preguntó si quería una mesa y el cliente le dijo que sí. El camarero le pre